In [6]:
import os
import pandas as pd
import numpy as np
import re
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from tensorflow.keras.utils import to_categorical


In [7]:
lines= pd.read_table('ben.txt', names=['eng', 'ben','word'])


In [8]:
english=[]
bengali=[]
for i in lines['eng']:
    english.append(i)
for j in lines["ben"]:
    bengali.append(j)

In [9]:
clean_english=[]
for word in english:
    word=re.sub('[?|!|.|]',"",word)
    word=word.lower()
    word=re.sub("I'm","I am",word)
    word=re.sub("can't","can not",word)
    word=re.sub("that's","that is",word)
    word=re.sub("we're","we are",word)
    word=re.sub("I'm","I am",word)
    word=re.sub("don't","do not",word)
    word=re.sub("we'll","we will",word)
    word=re.sub("tom's","tom is",word)
    word=re.sub("who'll","who will",word)
    word=re.sub("who's","who is",word)
    word=re.sub("i'll","i will",word)
    word=re.sub("it's","it is",word)
    word=re.sub("name's","name is",word)
    word=re.sub("she's","she is",word)
    word=re.sub("i'","i am",word)
    word=re.sub("won't","will not",word)
    clean_english.append(word)

In [10]:
clean_bengali=[]
for word in bengali:
    word=re.sub('[?|!|.|]',"",word)
    word=re.sub("।","",word)
    word="START_ "+word+" _END"
    clean_bengali.append(word)

In [11]:
len(clean_bengali)

4617

In [12]:
clean_bengali[:10]

['START_ যাও _END',
 'START_ যান _END',
 'START_ যা _END',
 'START_ পালাও _END',
 'START_ পালান _END',
 'START_ কে _END',
 'START_ আগুন _END',
 'START_ বাঁচাও _END',
 'START_ বাঁচান _END',
 'START_ থামুন _END']

In [13]:
all_words_bengali=[]
length_words=[]
for i in clean_bengali:
    for word in i.split(" "):
        length_words.append(len(i.split(" ")))
        if word not in all_words_bengali:
            all_words_bengali.append(word)
    

In [14]:
print("Total unique words in Bengali", len(all_words_bengali))
print("Maximum Length is",max(length_words))

Total unique words in Bengali 3071
Maximum Length is 20


In [15]:
all_words_english=[]
length_english=[]
for i in clean_english:
    for word in i.split(" "):
        length_english.append(len(i.split(" ")))
        if word not in all_words_english:
            all_words_english.append(word)
    

In [16]:
print("Total unique words in English", len(all_words_english))
print("Maximum Length is",max(length_english))

Total unique words in English 1971
Maximum Length is 19


In [17]:
english_token_index = dict([(word, i+1) for i, word in enumerate(all_words_english)])
bengali_token_index = dict([(word, i+1) for i, word in enumerate(all_words_bengali)])

In [18]:
reverse_english_char_index = dict((i, word) for word, i in english_token_index.items())
reverse_bengali_char_index = dict((i, word) for word, i in bengali_token_index.items())

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train,X_test,Y_train,Y_test=train_test_split(clean_english,clean_bengali,test_size=.15)

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Training Data

In [90]:
X=[]
Y=[]
output=[]
for i in range(len(X_train)):
    seq=[]
    for word in X_train[i].split(" "):
        seq.append(english_token_index[word])
    bengali_seq=[]
    for words in Y_train[i].split(" "):
        bengali_seq.append(bengali_token_index[words])
    x_pad=pad_sequences([seq],maxlen=19,padding="post")
    y_pad=pad_sequences([bengali_seq],maxlen=20,padding="post")
    se=np.zeros(3071)
    out=bengali_seq[1:]
    out_pad=pad_sequences([out],maxlen=20,padding="post")
    final_output = to_categorical(out_pad,3072)
    
    X.append(x_pad)
    Y.append(y_pad)
    output.append(final_output)
    

# Storing The Test Data

In [80]:
X_test_=[]
Y_test_=[]
output=[]
for i in range(len(X_test)):
    seq=[]
    for word in X_test[i].split(" "):
        seq.append(english_token_index[word])
   
    x_pad=pad_sequences([seq],maxlen=19,padding="post")

    X_test_.append(x_pad)
    Y_test_.append(Y_test[i])
    
    

In [96]:
x=[]
for i in X:
    x.append(i[0])

In [97]:
y=[]
for i in Y:
    y.append(i[0])

In [98]:
x=np.array(x)

In [99]:
y=np.array(y)

In [91]:
all_out=[]
for i in output:
    all_out.append(i[0])

In [100]:
x.shape

(3924, 19)

In [101]:
y.shape

(3924, 20)

In [94]:
output=np.array(all_out)

In [95]:
output.shape

(3924, 20, 3072)

# Encoder and Decoder Model

In [102]:
max_length_src=19
max_length_tar=20
latent_dim = 50
num_encoder_tokens=1972
num_decoder_tokens=3072

In [103]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [104]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [105]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [106]:
model.fit([x,y],output,batch_size=95,epochs=200)

Epoch 1/200
42/42 [==============================] - 16s 141ms/step - loss: 2.1500 - acc: 0.1528
Epoch 2/200
42/42 [==============================] - 6s 142ms/step - loss: 1.6856 - acc: 0.1580
Epoch 3/200
42/42 [==============================] - 6s 148ms/step - loss: 1.6115 - acc: 0.1721
Epoch 4/200
42/42 [==============================] - 6s 139ms/step - loss: 1.5458 - acc: 0.1767
Epoch 5/200
42/42 [==============================] - 6s 142ms/step - loss: 1.4936 - acc: 0.2750
Epoch 6/200
42/42 [==============================] - 6s 139ms/step - loss: 1.4463 - acc: 0.3279
Epoch 7/200
42/42 [==============================] - 6s 139ms/step - loss: 1.4036 - acc: 0.3459
Epoch 8/200
42/42 [==============================] - 6s 138ms/step - loss: 1.3663 - acc: 0.3540
Epoch 9/200
42/42 [==============================] - 6s 139ms/step - loss: 1.3339 - acc: 0.3580
Epoch 10/200
42/42 [==============================] - 6s 138ms/step - loss: 1.3062 - acc: 0.3624
Epoch 11/200
42/42 [==================

42/42 [==============================] - 6s 142ms/step - loss: 0.5094 - acc: 0.6720
Epoch 86/200
42/42 [==============================] - 6s 143ms/step - loss: 0.5032 - acc: 0.6744
Epoch 87/200
42/42 [==============================] - 6s 143ms/step - loss: 0.4971 - acc: 0.6788
Epoch 88/200
42/42 [==============================] - 6s 144ms/step - loss: 0.4907 - acc: 0.6840
Epoch 89/200
42/42 [==============================] - 6s 142ms/step - loss: 0.4847 - acc: 0.6863
Epoch 90/200
42/42 [==============================] - 6s 141ms/step - loss: 0.4787 - acc: 0.6911
Epoch 91/200
42/42 [==============================] - 4s 102ms/step - loss: 0.4731 - acc: 0.6935
Epoch 92/200
42/42 [==============================] - 4s 86ms/step - loss: 0.4669 - acc: 0.6970
Epoch 93/200
42/42 [==============================] - 4s 86ms/step - loss: 0.4614 - acc: 0.7009
Epoch 94/200
42/42 [==============================] - 4s 84ms/step - loss: 0.4552 - acc: 0.7049
Epoch 95/200
42/42 [==========================

42/42 [==============================] - 4s 100ms/step - loss: 0.1917 - acc: 0.8779
Epoch 169/200
42/42 [==============================] - 4s 102ms/step - loss: 0.1894 - acc: 0.8785
Epoch 170/200
42/42 [==============================] - 4s 101ms/step - loss: 0.1876 - acc: 0.8793
Epoch 171/200
42/42 [==============================] - 4s 100ms/step - loss: 0.1855 - acc: 0.8820
Epoch 172/200
42/42 [==============================] - 4s 100ms/step - loss: 0.1838 - acc: 0.8828
Epoch 173/200
42/42 [==============================] - 4s 105ms/step - loss: 0.1816 - acc: 0.8844
Epoch 174/200
42/42 [==============================] - 5s 117ms/step - loss: 0.1795 - acc: 0.8852
Epoch 175/200
42/42 [==============================] - 5s 110ms/step - loss: 0.1777 - acc: 0.8858
Epoch 176/200
42/42 [==============================] - 5s 111ms/step - loss: 0.1760 - acc: 0.8869
Epoch 177/200
42/42 [==============================] - 4s 102ms/step - loss: 0.1737 - acc: 0.8883
Epoch 178/200
42/42 [=============

# Encoder and Decoder for Inference 

In [107]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) 

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) 

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Prediction

In [108]:
for x in range(10):
    data=X_test_[x]
    state_value=encoder_model.predict(data)
    target_seq=np.zeros((1,1))
    target_seq[0,0]=bengali_token_index["START_"]
    
    Flag=False
    predicted_sentence=" "
    while not Flag:
        output,h,c=decoder_model.predict([target_seq]+state_value)
        index=np.argmax(output[0][0])
        char=reverse_bengali_char_index[index]
        predicted_sentence=predicted_sentence+" "+char
        if char=="_END" or len(predicted_sentence)>19:
            Flag=True
        target_seq=np.zeros((1,1))
        target_seq[0,0]=index
        state_value=[h,c]
    print("English::", X_test[x])
    print("Original-Bengali::", Y_test[x])
    print("Predicted Bengali::", predicted_sentence)
    print()

English:: i can not lift my right arm
Original-Bengali:: START_ আমি আমার ডান হাতটা তুলতে পারছি না _END
Predicted Bengali::   আমার ক্ষমা আর না আমার

English:: we are both right
Original-Bengali:: START_ আমরা দুজনেই ঠিক _END
Predicted Bengali::   আমরা যা করব _END

English:: i will call first
Original-Bengali:: START_ আমি আগে ফোন করবো _END
Predicted Bengali::   আমি তাদের পছন্দ করি

English:: my hip hurts
Original-Bengali:: START_ আমার কোমর ব্যাথা করছে _END
Predicted Bengali::   আমার মালপত্রটা ব্যাথা

English:: tom is hurt
Original-Bengali:: START_ টম আহত _END
Predicted Bengali::   টম আহত _END

English:: tom shouted
Original-Bengali:: START_ টম চেঁচালো _END
Predicted Bengali::   টম চিৎকার করলো _END

English:: tom was reading the newspaper when mary walked in
Original-Bengali:: START_ টম যখন খবরের কাগজ পড়ছিল তখন মেরি ঘরে ঢুকল _END
Predicted Bengali::   টম মেরি কাউকে মেরি

English:: i know tom was a friend of yours
Original-Bengali:: START_ আমি জানি টম আপনার বন্ধু ছিলেন _END
Predicted Benga